In [1]:
#%pip install -r ./requirements.txt

In [2]:
from tonic.datasets.nmnist import NMNIST
from tonic.transforms import ToFrame

root_dir = "./NMNIST"
NMNIST(save_to=root_dir, train=True)
NMNIST(save_to=root_dir, train=False)

to_raster = ToFrame(sensor_size=(34, 34), n_time_bins=100)
dataset = NMNIST(save_to=root_dir, train=False, transform=to_raster)


/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch import nn

cnn = nn.Sequential(
    nn.Conv2d(2, 8, 3, padding=1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2),
    nn.Conv2d(8, 16, 3, padding=1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2),
    nn.Conv2d(16, 16, 3, padding=1, stride=2, bias=False),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(256, 10, bias=False),
    nn.ReLU()
)


In [4]:
from sinabs.from_torch import from_model

snn_convert = from_model(model=cnn, input_shape=(2, 34, 34), batch_size=4).spiking_model


In [7]:
from sinabs.backend.dynapcnn import DynapcnnNetwork

cpu_snn = snn_convert.to("cpu")
dynapcnn = DynapcnnNetwork(snn=cpu_snn, input_shape=(2, 34, 34), discretize=True, dvs_input=False)
dynapcnn.to(device="speck2fmodule", chip_layers_ordering="auto")


Network is valid


KeyError: 'speck2fmodule:0'

In [ ]:
from sinabs.backend.dynapcnn.dynapcnn_visualizer import DynapcnnVisualizer

visualizer = DynapcnnVisualizer(window_scale=(4, 8), dvs_shape=(34, 34), spike_collection_interval=50)
visualizer.connect(dynapcnn)


In [ ]:
dvs_event = samna.speck2f.event.DvsEvent()
# 设置 dvs_event 的属性：x, y, t, p
# 然后发送到 devkit 推理
output = dynapcnn([dvs_event])


In [11]:
import samna
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnNetwork

# 打开设备
devices = samna.device.get_unopened_devices()
if len(devices) == 0:
    raise Exception("No devices found")

# 打开第一个设备
my_board = samna.device.open_device(devices[0])

# 获取设备的 serial_number
device_id = my_board.get_serial_number()  # 获取设备的 serial_number 作为设备 ID

# 你的 SNN 模型，例如通过 CNN -> SNN 转换得到
snn_model = from_model(
    model=cnn,  # 你的 CNN 模型
    input_shape=(2, 34, 34),
    batch_size=4
).spiking_model

# 包装为 DynapcnnNetwork
dynapcnn = DynapcnnNetwork(
    snn=snn_model.to("cpu"),
    input_shape=(2, 34, 34),
    discretize=True,
    dvs_input=False
)

# ✅ 使用 serial_number 作为设备标识符
dynapcnn.to(device=device_id, chip_layers_ordering="auto")

print(f"✅ 模型成功部署到 SPECK！部署到核心顺序为: {dynapcnn.chip_layers_ordering}")


AttributeError: 'samna.speck2fBoards.Speck2fDevKit' object has no attribute 'get_serial_number'